# Focault test data reduction

This document will describe step by step the process of extracting the surface profile of a telescope mirror from Foucault or Wire test data.

## Modeling the data

Before jumping into the simulation, we need to find the model that will obtain the mirror surface profile from test data.
After that, we will perform a simulation with real world data.

### Derivation of the equations

First we need to derive the set of equations that will alow to obtaing the mirror surface profile from test data.

For a generic concave mirror surface, the Foucault or Wire tests will show for each mirror radius the equivalent radius of curvature it has. This can be interpreted as finding a succession of spherical surfaces that approximate the existing mirror surface (dashed curve in figure). Obviously that for a completely spherical mirror surface, there is exactly one sphere that matches the whole surface.

<img src="figures/diagram_1.svg"/>

The result of these tests is a set of data pairs consisting of mirror diameter $x$ (independent variable) and its respective radius of curvature $f(x)$ (dependent variable).

To help formulate the equations that model these tests, we can instead think of vectors $G(x)$ and $F(x)$ representing surface height respective to its center and radius of curvature respectively:

$G(x) = \langle x, g(x) \rangle$

$F(x) = \langle 0, f(x) \rangle$

Subtracting the two vectors gives:

$F(x) - G(x) = \langle 0 - x, f(x)-g(x) \rangle = \langle -x, f(x) - g(x) \rangle$

If we then represent the tangent vector to mirror surface at $G(x)$:

$T(x) = \frac{G'(x)}{|G'(x)|} = \langle\frac{1}{|G'(x)|}, \frac{z'(x)}{|G'(x)|}\rangle$

Then it follows that:

$T(x) \cdot (F(x) - G(x)) = 0$

$\left( - \frac{x}{|G'(x)|} + (f(x) - g(x))\frac{g'(x)}{|G'(x)|} \right) = 0$

$(f(x) - g(x))g'(x) = x$

This results in the following differential equation:

$g'(x) = \frac{x}{f(x) - g(x)}$

Knowing the data pairs $x$ and $f(x)$, we can integrate the equation above and get the mirror surface profile.

### Finding a solution

Now in order to ... tbd Runge Kutta

## Real world example

The real world collected data that will be used in this exercise is stored in a JSON file.

First we need to read the test data from the JSON file.

In [1]:
import json

with open('mirror_measurements_database.json', "r") as data_base_file:
    data_base = json.load(data_base_file)
    data_base_file.close()

radius_of_curvature = data_base['radius_of_curvature_measurement']['radius_of_curvature']
measurements = sorted(data_base['measurements'], key=lambda m: m['optical_axis_offset'])

For convenience and to allow later use of the SciPy mathematical functions, we will first transform the data into Numpy arrays.

In [2]:
import pandas

measurements_data_frame = pandas.DataFrame(measurements)

display(measurements_data_frame) # Module display is part of library IPython

measurements_data_frame = measurements_data_frame.iloc[:,:2] # Remove the column containing the Foucault image captures

measurements_matrix = measurements_data_frame.to_numpy()
x = measurements_matrix[:, 1]
f = measurements_matrix[:, 0]

optical_axis_offset  mirror_radius foucaultgram
0                  0.50          43.08       na.jpg
1                  0.60          47.55       na.jpg
2                  0.70          51.57       na.jpg
3                  0.80          55.37       na.jpg
4                  0.90          61.65       na.jpg
5                  1.00          64.06       na.jpg
6                  1.10          67.20       na.jpg
7                  1.20          69.21       na.jpg
8                  1.30          72.82       na.jpg
9                  1.40          74.63       na.jpg
10                 1.45          76.21       na.jpg
11                 1.50          77.88       na.jpg
12                 1.55          78.59       na.jpg
13                 1.60          80.61       na.jpg
14                 1.65          82.12       na.jpg
15                 1.70          83.96       na.jpg
16                 1.75          85.48       na.jpg
17                 1.80          87.03       na.jpg
18                 1.85          88.79       na.jpg
19                 1.90          90.43       na.jpg
20                 1.95          92.99       na.jpg
21                 2.00          95.38       na.jpg

kjhkjhkh

In [3]:
import numpy
from scipy import interpolate

interpolate_f = interpolate.interp1d(x, f, kind="cubic")

num_samples = int((x[-1] - x[0])*2)

x_e = numpy.linspace(x[0], x[-1], num=num_samples)
f_e = interpolate_f(x_e)

kjhkhkjh

In [4]:
%matplotlib notebook
from matplotlib import pyplot

pyplot.scatter(x, f, label="Raw test data")
pyplot.plot(x_e, f_e, "g-", label="Raw test data (interpolated)")
pyplot.xlabel('Mirror radius [mm]')
pyplot.ylabel('Foucault/Wire test offsets [mm]')
pyplot.show()

<IPython.core.display.Javascript object>

In [5]:
from numba import jit

@jit(nopython=True)
def mirror_slope_ode_generic(x, f, g):
    dg = numpy.divide(x, numpy.subtract(f, g))
    return dg

def mirror_slope_ode(x, g):
    f_t = radius_of_curvature + interpolate_f(x)
    dg = mirror_slope_ode_generic(x, f_t, g)
    return dg

In [6]:
from scipy.integrate import solve_ivp

ode_solution = solve_ivp(fun=mirror_slope_ode,
                         t_span=[x_e[0], x_e[-1]],
                         y0=[0.0],
                         method="RK45",
                         t_eval=x_e,
                         vectorized=True,
                         dense_output=True)

In [7]:
g = numpy.reshape(ode_solution.y, (-1))

print(g)

[0.         0.01031817 0.02075725 0.03131722 0.04199809 0.05279986
 0.06372254 0.07476611 0.08593059 0.09721597 0.10862226 0.12014944
 0.13179752 0.1435665  0.15545639 0.16746717 0.17959885 0.19185142
 0.20422489 0.21671926 0.22933452 0.24207067 0.2549277  0.26790567
 0.28100458 0.29422444 0.30756525 0.32102698 0.33460966 0.34831326
 0.3621378  0.37608325 0.39014963 0.40433693 0.41864515 0.43307428
 0.44762432 0.46229527 0.47708713 0.49199989 0.50703356 0.52218813
 0.5374636  0.55285996 0.56837723 0.58401539 0.59977445 0.6156544
 0.63165524 0.64777697 0.6640196  0.68038312 0.69686753 0.71347283
 0.73019902 0.7470461  0.76401407 0.78110293 0.79831269 0.81564334
 0.83309488 0.85066731 0.86836064 0.88617486 0.90410998 0.922166
 0.94034292 0.95864074 0.97705946 0.99559909 1.01425962 1.03304106
 1.05194342 1.07096669 1.09011087 1.10937597 1.12876199 1.14826894
 1.16789682 1.18764562 1.20751536 1.22750603 1.24761765 1.26785021
 1.28820371 1.30867817 1.32927359 1.34998996 1.3708273  1.3917856

In [8]:
print(ode_solution.message)

The solver successfully reached the end of the integration interval.
